In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<font color='tomato'><font color="#CC3D3D"><p>
# A DNN Model for Multiclass Classification

##### Import modules

In [ ]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 133 kB 8.6 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import os
import random
import pickle
from tqdm import tqdm
from IPython.display import Image, clear_output
import seaborn as sns
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
%matplotlib inline

from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
import kerastuner as kt
print(tf.__version__)

2.8.2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  app.launch_new_instance()


##### Set random seeds to make your results reproducible

In [ ]:
# 매번 모델링을 할 때마다 동일한 결과를 얻으려면 아래 코드를 실행해야 함.

def reset_seeds(s1,s2,s3, reset_graph_with_backend=None):
    if reset_graph_with_backend is not None:
        K = reset_graph_with_backend
        K.clear_session()
        tf.compat.v1.reset_default_graph()
        print("KERAS AND TENSORFLOW GRAPHS RESET")  # optional

    np.random.seed(s1)
    random.seed(s2)
    tf.compat.v1.set_random_seed(s3)
    os.environ['CUDA_VISIBLE_DEVICES'] = ''  # for GPU
#    print("RANDOM SEEDS RESET")  # optional

### Step 1: Load and process the data

##### Read data

In [ ]:
# 학습데이터에 custid가 없으면 추가
IDtest = pd.DataFrame({'custid': pd.read_csv('/content/drive/MyDrive/머러컴피티션/X_test.csv', encoding = "cp949").custid.unique()})

In [ ]:
# 데이터불러오기

In [ ]:
df_train = pd.read_csv('snd_sum_shap_0615.csv')

In [ ]:
df_test = pd.read_csv('snd_sum_shap_te_0615.csv')

In [ ]:
y_train = pd.read_csv('/content/drive/MyDrive/머러컴피티션/y_train.csv').group

##### One-hot-encode Target variable 

In [ ]:
# 8개의 범주형 타겟 값을 one-hot-encoding을 통해 8개의 컬럼으로 만들어야 함.
y_train = keras.utils.to_categorical(y_train.astype('category').cat.codes)

In [ ]:
df_train

,1,0,169,231,4,3,2,173,352,25,...,839,34,841,1320,1155,911,873,1345,535,1141
0,-0.401774,0.258656,0.616024,0.242885,0.059594,0.008479,1.098612,-0.002424,-0.517807,19.000000,...,-0.107617,-0.152542,0,-0.217619,0.0,-0.196841,-0.083853,-0.173885,-0.218596,-0.078677
1,-0.347583,0.258891,0.380067,0.241397,0.028305,0.055183,1.098612,-0.010025,0.272619,14.142857,...,-0.107617,9.593732,0,-0.217619,0.0,-0.196841,-0.083853,-0.173885,-0.218596,-0.078677
2,-0.017115,0.260489,-0.710264,0.234344,-0.003134,-0.086416,1.098612,0.015999,-0.227984,16.000000,...,-0.107617,-0.152542,0,-0.217619,0.0,-0.196841,-0.083853,-0.173885,0.164404,-0.078677
3,0.103395,0.260940,-1.058357,0.232445,-0.033140,-0.061537,0.693147,0.027703,-0.517807,14.000000,...,-0.107617,-0.152542,0,-0.217619,0.0,-0.196841,-0.083853,-0.173885,-0.218596,-0.078677
4,-0.456512,0.258369,0.827803,0.244247,0.032277,0.086695,1.386294,0.016045,0.840738,15.444444,...,-0.107617,-0.152542,0,-0.217619,0.0,-0.196841,-0.083853,-0.173885,-0.218596,-0.078677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21582,-0.260748,0.259313,0.315357,0.241041,0.003974,-0.072648,1.098612,0.010104,-0.517807,15.060000,...,-0.107617,-0.152542,0,-0.217619,5.0,11.223432,-0.083853,-0.173885,-0.218596,-0.078677
21583,-1.113412,0.254983,0.261953,0.240611,-0.072966,-0.076272,0.693147,-0.029911,-0.517807,14.625000,...,-0.107617,-0.152542,6,-0.217619,0.0,-0.196841,-0.083853,-0.173885,-0.218596,-0.078677
21584,0.570301,0.263636,-0.537937,0.236095,-0.010159,-0.035039,1.098612,0.003240,-0.517807,13.500000,...,-0.107617,-0.152542,0,-0.217619,1.0,-0.196841,-0.083853,-0.173885,-0.218596,-0.078677
21585,-0.745471,0.256832,0.384082,0.241566,-0.013758,-0.121351,0.693147,0.024548,-0.517807,14.500000,...,-0.107617,-0.152542,0,-0.217619,0.0,-0.196841,-0.083853,-0.173885,-0.218596,-0.078677


In [ ]:
df_trains = df_train.iloc[:,1:]
df_tests = df_test.iloc[:,1:]

In [ ]:
X_train = df_train
X_test = df_test

##### Split data into train & validation set 

In [ ]:
i = int(round(X_train.shape[0] * 0.8,0))
X_valid, y_valid = X_train[i:], y_train[i:]
X_train, y_train = X_train[:i], y_train[:i]

In [ ]:
y_train

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

##### Feature scaling

In [ ]:
X_train.shape, X_valid.shape, X_test.shape

((17270, 1617), (4317, 1617), (14380, 1617))

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

### Step 2: Define the hyper-model

In [ ]:
def model_fn(hp):
    inputs = keras.Input(shape=(X_train.shape[1],))
    x = inputs
    for i in range(hp.Int('num_layers', 2, 3)):
        x = keras.layers.Dense(hp.Int('unit_'+str(i), 16, 64, step=16), activation='relu')(x)
        x = keras.layers.Dropout(hp.Float('dropout_'+str(i), 0, 0.5, step=0.25, default=0.5))(x)
    outputs = keras.layers.Dense(8, activation='softmax')(x) # 예측값이 8종류이므로 8개 출력 뉴런 필요
    model = keras.Model(inputs, outputs)
    model.compile(loss='categorical_crossentropy', # Multiclass Classification에서 사용하는 loss function
                  optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), 
                  metrics=[keras.metrics.CategoricalCrossentropy()]) # Multiclass Classification에서 사용하는 평가지표
    return model

### Step 3: Build multiple hyper-tuned models

In [ ]:
# 
tuner = kt.Hyperband(model_fn,
                     objective=kt.Objective('val_categorical_crossentropy', direction="min"), 
                     max_epochs=20,
                     hyperband_iterations=2,
                     overwrite=True,
                     directory='dnn_tuning')
tuner.search(X_train, y_train, validation_data=(X_valid, y_valid), 
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])
model = tuner.get_best_models(1)[0]  
tuner.results_summary(1)

Trial 60 Complete [00h 00m 58s]
val_categorical_crossentropy: 1.5927852392196655

Best val_categorical_crossentropy So Far: 1.4924216270446777
Total elapsed time: 00h 15m 07s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in dnn_tuning/untitled_project
Showing 1 best trials
Trial summary
Hyperparameters:
num_layers: 2
unit_0: 48
dropout_0: 0.25
unit_1: 48
dropout_1: 0.25
learning_rate: 0.001
unit_2: 64
dropout_2: 0.25
tuner/epochs: 7
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 1.4924216270446777


In [ ]:
t = pd.Timestamp.now()
fname = f"dnn_submission_{t.month:02}{t.day:02}.1.492.csv"
pred = pd.DataFrame(model.predict(X_test))
pred.columns = ['F20','F30','F40','F50','M20','M30','M40','M50']

pd.concat([pd.Series(IDtest.custid, name="ID"), pred] ,axis=1).to_csv('/content/drive/MyDrive/머러컴피티션/진짜마지막/dnn결과/'+fname, index=False)
print(f"'{fname}' is ready to submit.")

'dnn_submission_0616.1.492.csv' is ready to submit.


In [ ]:
# 
tuner = kt.Hyperband(model_fn,
                     objective=kt.Objective('val_categorical_crossentropy', direction="min"), 
                     max_epochs=20,
                     hyperband_iterations=2,
                     overwrite=True,
                     directory='dnn_tuning')
tuner.search(X_train, y_train, validation_data=(X_valid, y_valid), 
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])
model = tuner.get_best_models(1)[0]  
tuner.results_summary(1)

Trial 60 Complete [00h 00m 40s]
val_categorical_crossentropy: 1.6262036561965942

Best val_categorical_crossentropy So Far: 1.4815562963485718
Total elapsed time: 00h 17m 08s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in dnn_tuning/untitled_project
Showing 1 best trials
Trial summary
Hyperparameters:
num_layers: 2
unit_0: 64
dropout_0: 0.5
unit_1: 48
dropout_1: 0.0
learning_rate: 0.001
unit_2: 48
dropout_2: 0.5
tuner/epochs: 7
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 1.4815562963485718


In [ ]:
model = tuner.get_best_models(1)[0]  
tuner.results_summary(1)

IndexError: ignored

In [ ]:
t = pd.Timestamp.now()
fname = f"dnn_submission_{t.month:02}{t.day:02}.1.481.csv"
pred = pd.DataFrame(model.predict(X_test))
pred.columns = ['F20','F30','F40','F50','M20','M30','M40','M50']

pd.concat([pd.Series(IDtest.custid, name="ID"), pred] ,axis=1).to_csv('/content/drive/MyDrive/머러컴피티션/submission/snd_sum_shap_0612.csv/'+fname, index=False)
print(f"'{fname}' is ready to submit.")

'dnn_submission_0616.1.481.csv' is ready to submit.


In [ ]:
# 
tuner = kt.Hyperband(model_fn,
                     objective=kt.Objective('val_categorical_crossentropy', direction="min"), 
                     max_epochs=20,
                     hyperband_iterations=2,
                     overwrite=True,
                     directory='dnn_tuning')
tuner.search(X_train, y_train, validation_data=(X_valid, y_valid), 
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])
model = tuner.get_best_models(1)[0]  
tuner.results_summary(1)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
2                 |?                 |num_layers
48                |?                 |unit_0
0                 |?                 |dropout_0
64                |?                 |unit_1
0.25              |?                 |dropout_1
0.01              |?                 |learning_rate
3                 |?                 |tuner/epochs
0                 |?                 |tuner/initial_epoch
2                 |?                 |tuner/bracket
0                 |?                 |tuner/round

Epoch 1/3
 30/540 [>.............................] - ETA: 1s - loss: 2.3839 - categorical_crossentropy: 2.3839

KeyboardInterrupt: ignored

In [ ]:
t = pd.Timestamp.now()
fname = f"dnn_submission_{t.month:02}{t.day:02}.1.490.csv"
pred = pd.DataFrame(model.predict(X_test))
pred.columns = ['F20','F30','F40','F50','M20','M30','M40','M50']

pd.concat([pd.Series(IDtest.custid, name="ID"), pred] ,axis=1).to_csv('/content/drive/MyDrive/머러컴피티션/submission/snd_sum_shap_0612.csv/'+fname, index=False)
print(f"'{fname}' is ready to submit.")

In [ ]:
dnn487.to_csv('/content/drive/MyDrive/머러컴피티션/submission/snd_sum_shap_0612.csv/'+'dnn487', index=False)

In [ ]:
# second tuning 전 step=0.25,max_epochs=20, num 4 3 16 48 6
tuner = kt.Hyperband(model_fn,
                     objective=kt.Objective('val_categorical_crossentropy', direction="min"), 
                     max_epochs=20,
                     hyperband_iterations=2,
                     overwrite=True,
                     directory='dnn_tuning')
tuner.search(X_train, y_train, validation_data=(X_valid, y_valid), 
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])
model = tuner.get_best_models(1)[0]  
tuner.results_summary(1)

Trial 60 Complete [00h 00m 52s]
val_categorical_crossentropy: 1.530590295791626

Best val_categorical_crossentropy So Far: 1.4903638362884521
Total elapsed time: 00h 11m 44s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in dnn_tuning/untitled_project
Showing 1 best trials
Trial summary
Hyperparameters:
num_layers: 3
unit_0: 48
dropout_0: 0.25
unit_1: 64
dropout_1: 0.25
unit_2: 48
dropout_2: 0.0
learning_rate: 0.001
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0022
Score: 1.4903638362884521


In [ ]:
t = pd.Timestamp.now()
fname = f"dnn_submission_{t.month:02}{t.day:02}.1.490.csv"
pred = pd.DataFrame(model.predict(X_test))
pred.columns = ['F20','F30','F40','F50','M20','M30','M40','M50']

In [ ]:

pd.concat([pd.Series(IDtest.custid, name="ID"), pred] ,axis=1).to_csv('/content/drive/MyDrive/머러컴피티션/submission/snd_sum_shap_0612.csv/'+fname, index=False)
print(f"'{fname}' is ready to submit.")

'dnn_submission_0615.1.490.csv' is ready to submit.


In [ ]:
# second tuning 전 step=0.25,max_epochs=20, num 3 80
tuner = kt.Hyperband(model_fn,
                     objective=kt.Objective('val_categorical_crossentropy', direction="min"), 
                     max_epochs=20,
                     hyperband_iterations=2,
                     overwrite=True,
                     directory='dnn_tuning')
tuner.search(X_train, y_train, validation_data=(X_valid, y_valid), 
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])
model = tuner.get_best_models(1)[0]  
tuner.results_summary(1)

Trial 60 Complete [00h 00m 20s]
val_categorical_crossentropy: 1.502875566482544

Best val_categorical_crossentropy So Far: 1.48936927318573
Total elapsed time: 00h 11m 10s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in dnn_tuning/untitled_project
Showing 1 best trials
Trial summary
Hyperparameters:
num_layers: 3
unit_0: 48
dropout_0: 0.25
unit_1: 64
dropout_1: 0.0
unit_2: 48
dropout_2: 0.25
learning_rate: 0.001
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0042
Score: 1.48936927318573


In [ ]:
t = pd.Timestamp.now()
fname = f"dnn_submission_{t.month:02}{t.day:02}.1.489.csv"
pred = pd.DataFrame(model.predict(X_test))
pred.columns = ['F20','F30','F40','F50','M20','M30','M40','M50']
pd.concat([pd.Series(IDtest.custid, name="ID"), pred] ,axis=1).to_csv('/content/drive/MyDrive/머러컴피티션/submission/snd_sum_shap_0612.csv/'+fname, index=False)
print(f"'{fname}' is ready to submit.")

'dnn_submission_0615061.489.csv' is ready to submit.


In [ ]:
# second tuning 전 step=0.25,max_epochs=20,
tuner = kt.Hyperband(model_fn,
                     objective=kt.Objective('val_categorical_crossentropy', direction="min"), 
                     max_epochs=20,
                     hyperband_iterations=2,
                     overwrite=True,
                     directory='dnn_tuning')
tuner.search(X_train, y_train, validation_data=(X_valid, y_valid), 
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])
model = tuner.get_best_models(1)[0]  
tuner.results_summary(1)

Trial 60 Complete [00h 00m 09s]
val_categorical_crossentropy: 1.566907525062561

Best val_categorical_crossentropy So Far: 1.4984831809997559
Total elapsed time: 00h 10m 24s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in dnn_tuning/untitled_project
Showing 1 best trials
Trial summary
Hyperparameters:
num_layers: 2
unit_0: 32
dropout_0: 0.25
unit_1: 32
dropout_1: 0.25
learning_rate: 0.001
unit_2: 48
dropout_2: 0.5
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0043
Score: 1.4984831809997559


In [ ]:
# second tuning 전 step=0.2,max_epochs=20,
tuner = kt.Hyperband(model_fn,
                     objective=kt.Objective('val_categorical_crossentropy', direction="min"), 
                     max_epochs=20,
                     hyperband_iterations=2,
                     overwrite=True,
                     directory='dnn_tuning')
tuner.search(X_train, y_train, validation_data=(X_valid, y_valid), 
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])
model = tuner.get_best_models(1)[0]  
tuner.results_summary(1)

Trial 60 Complete [00h 00m 06s]
val_categorical_crossentropy: 1.695425271987915

Best val_categorical_crossentropy So Far: 1.493905782699585
Total elapsed time: 00h 10m 19s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in dnn_tuning/untitled_project
Showing 1 best trials
Trial summary
Hyperparameters:
num_layers: 2
unit_0: 48
dropout_0: 0.4
unit_1: 16
dropout_1: 0.2
learning_rate: 0.001
unit_2: 32
dropout_2: 0.2
tuner/epochs: 7
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 1.493905782699585


In [ ]:
# second tuning 전 step=0.25,max_epochs=20,
tuner = kt.Hyperband(model_fn,
                     objective=kt.Objective('val_categorical_crossentropy', direction="min"), 
                     max_epochs=20,
                     hyperband_iterations=2,
                     overwrite=True,
                     directory='dnn_tuning')
tuner.search(X_train, y_train, validation_data=(X_valid, y_valid), 
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])
model = tuner.get_best_models(1)[0]  
tuner.results_summary(1)

Trial 60 Complete [00h 00m 05s]
val_categorical_crossentropy: 1.5981625318527222

Best val_categorical_crossentropy So Far: 1.4822351932525635
Total elapsed time: 00h 09m 06s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in dnn_tuning/untitled_project
Showing 1 best trials
Trial summary
Hyperparameters:
num_layers: 2
unit_0: 64
dropout_0: 0.5
unit_1: 48
dropout_1: 0.0
learning_rate: 0.001
unit_2: 16
dropout_2: 0.25
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0042
Score: 1.4822351932525635


In [ ]:
# second tuning 후 step=0.25,max_epochs=20,
tuner = kt.Hyperband(model_fn,
                     objective=kt.Objective('val_categorical_crossentropy', direction="min"), 
                     max_epochs=20,
                     hyperband_iterations=2,
                     overwrite=True,
                     directory='dnn_tuning')
tuner.search(X_train, y_train, validation_data=(X_valid, y_valid), 
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])
model = tuner.get_best_models(1)[0]  
tuner.results_summary(1)

Trial 60 Complete [00h 00m 18s]
val_categorical_crossentropy: 1.5091098546981812

Best val_categorical_crossentropy So Far: 1.491424560546875
Total elapsed time: 00h 08m 21s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in dnn_tuning/untitled_project
Showing 1 best trials
Trial summary
Hyperparameters:
num_layers: 2
unit_0: 16
dropout_0: 0.2
unit_1: 16
dropout_1: 0.2
learning_rate: 0.001
unit_2: 16
dropout_2: 0.4
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0023
Score: 1.491424560546875


In [ ]:
# second tuning 후 256
tuner = kt.Hyperband(model_fn,
                     objective=kt.Objective('val_categorical_crossentropy', direction="min"), 
                     max_epochs=10,
                     hyperband_iterations=2,
                     overwrite=True,
                     directory='dnn_tuning')
tuner.search(X_train, y_train, validation_data=(X_valid, y_valid), 
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])
model = tuner.get_best_models(1)[0]  
tuner.results_summary(1)

Trial 60 Complete [00h 00m 15s]
val_categorical_crossentropy: 1.4941835403442383

Best val_categorical_crossentropy So Far: 1.4926292896270752
Total elapsed time: 00h 07m 42s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in dnn_tuning/untitled_project
Showing 1 best trials
Trial summary
Hyperparameters:
num_layers: 5
unit_0: 224
dropout_0: 0.2
unit_1: 208
dropout_1: 0.2
learning_rate: 0.0001
unit_2: 64
dropout_2: 0.1
unit_3: 208
dropout_3: 0.0
unit_4: 176
dropout_4: 0.1
unit_5: 96
dropout_5: 0.4
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0012
Score: 1.4926292896270752


In [ ]:
# 1st tuning
tuner = kt.Hyperband(model_fn,
                     objective=kt.Objective('val_categorical_crossentropy', direction="min"), 
                     max_epochs=20,
                     hyperband_iterations=2,
                     overwrite=True,
                     directory='dnn_tuning')
tuner.search(X_train, y_train, validation_data=(X_valid, y_valid), 
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])
model = tuner.get_best_models(1)[0]  
tuner.results_summary(1)

Trial 60 Complete [00h 00m 04s]
val_categorical_crossentropy: 1.7119332551956177

Best val_categorical_crossentropy So Far: 1.4861210584640503
Total elapsed time: 00h 08m 21s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in dnn_tuning/untitled_project
Showing 1 best trials
Trial summary
Hyperparameters:
num_layers: 5
unit_0: 224
dropout_0: 0.2
unit_1: 208
dropout_1: 0.2
learning_rate: 0.0001
unit_2: 64
dropout_2: 0.1
unit_3: 208
dropout_3: 0.0
unit_4: 176
dropout_4: 0.1
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0043
Score: 1.4861210584640503


In [ ]:
X_train

array([[-0.4313704 , -0.43976308,  0.66140269, ..., -0.49763251,
        -0.22898994,  0.04961078],
       [-0.3807711 , -0.38066675,  0.66140269, ..., -0.54307493,
        -0.22898994,  0.04199081],
       [-0.03608654, -0.02028264,  0.66140269, ...,  0.06156525,
        -0.22898994, -0.63616637],
       ...,
       [-0.19590416, -0.20739021, -0.92168023, ..., -0.25070737,
        -0.22898994,  0.33706808],
       [-1.00836984, -1.02581696, -0.92168023, ..., -0.80171269,
        -0.22898994,  0.45218674],
       [-0.8884726 , -0.88631277,  0.66140269, ...,  3.99275047,
         2.51577423, -0.10468711]])

In [ ]:
t = pd.Timestamp.now()
fname = f"dnn_submission_{t.month:02}{t.day:02}{t.hour:02}1.493.csv"
pred = pd.DataFrame(model.predict(X_test))
pred.columns = ['F20','F30','F40','F50','M20','M30','M40','M50']
pd.concat([pd.Series(IDtest.custid, name="ID"), pred] ,axis=1).to_csv('/content/drive/MyDrive/머러컴피티션/submission/snd_sum_shap_0612.csv/'+fname, index=False)
print(f"'{fname}' is ready to submit.")

'dnn_submission_0615051.493.csv' is ready to submit.


In [ ]:
pd.read_csv('/content/drive/MyDrive/머러컴피티션/submission/dnn+lgbm/dnn_submission_06131301.csv')

,ID,F20,F30,F40,F50,M20,M30,M40,M50
0,30001,0.183698,0.138087,0.037649,0.116351,0.123118,0.209439,0.050899,0.140758
1,30002,0.005879,0.047781,0.325411,0.010709,0.001529,0.017993,0.572350,0.018347
2,30003,0.717163,0.186987,0.018404,0.017040,0.022689,0.023989,0.005746,0.007980
3,30005,0.513325,0.112715,0.106024,0.037476,0.065985,0.126673,0.016714,0.021089
4,30007,0.917493,0.060677,0.002505,0.002173,0.007393,0.007921,0.000681,0.001157
...,...,...,...,...,...,...,...,...,...
14375,49988,0.070449,0.443913,0.020704,0.007512,0.004949,0.367313,0.080524,0.004636
14376,49990,0.448465,0.257474,0.149285,0.020725,0.032518,0.051643,0.009137,0.030754
14377,49992,0.392092,0.126545,0.054908,0.073504,0.098653,0.178131,0.036839,0.039328
14378,49993,0.024272,0.649988,0.140282,0.004214,0.002694,0.129725,0.046946,0.001879


In [ ]:
pd.read_csv('/content/drive/MyDrive/머러컴피티션/submission/dnn+lgbm/dnn_submission_0613_1.512.csv')

,ID,F20,F30,F40,F50,M20,M30,M40,M50
0,30001,0.183698,0.138087,0.037649,0.116351,0.123118,0.209439,0.050899,0.140758
1,30002,0.005879,0.047781,0.325411,0.010709,0.001529,0.017993,0.572350,0.018347
2,30003,0.717163,0.186987,0.018404,0.017040,0.022689,0.023989,0.005746,0.007980
3,30005,0.513325,0.112715,0.106024,0.037476,0.065985,0.126673,0.016714,0.021089
4,30007,0.917493,0.060677,0.002505,0.002173,0.007393,0.007921,0.000681,0.001157
...,...,...,...,...,...,...,...,...,...
14375,49988,0.070449,0.443913,0.020704,0.007512,0.004949,0.367313,0.080524,0.004636
14376,49990,0.448465,0.257474,0.149285,0.020725,0.032518,0.051643,0.009137,0.030754
14377,49992,0.392092,0.126545,0.054908,0.073504,0.098653,0.178131,0.036839,0.039328
14378,49993,0.024272,0.649988,0.140282,0.004214,0.002694,0.129725,0.046946,0.001879


In [ ]:
pd.concat([pd.Series(IDtest.custid, name="ID"), pred] ,axis=1)

,ID,F20,F30,F40,F50,M20,M30,M40,M50
0,30001,0.127368,0.127141,0.070802,0.215573,0.076733,0.153606,0.043525,0.185252
1,30002,0.003001,0.047245,0.472640,0.029603,0.000180,0.016012,0.351101,0.080218
2,30003,0.782996,0.143690,0.021967,0.012854,0.013883,0.015447,0.002693,0.006471
3,30005,0.753075,0.105907,0.028009,0.022648,0.051321,0.023162,0.004991,0.010888
4,30007,0.886505,0.057098,0.005901,0.007949,0.031386,0.007237,0.000590,0.003335
...,...,...,...,...,...,...,...,...,...
14375,49988,0.178404,0.602676,0.026653,0.005637,0.007521,0.163109,0.011531,0.004470
14376,49990,0.455819,0.259807,0.094585,0.043054,0.029502,0.077526,0.014949,0.024758
14377,49992,0.328283,0.147437,0.089196,0.109694,0.114513,0.106707,0.031770,0.072399
14378,49993,0.046496,0.657718,0.036445,0.004434,0.002994,0.204592,0.042543,0.004780


<font color="#CC3D3D"><p>
# End